<a href="https://colab.research.google.com/github/maikenberthelsen/Proj1/blob/master/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function


from google.colab import drive
drive.mount('/content/drive/')

import gzip
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import glob
import shutil
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from mask_to_submission import *
from helpers import *
from F1_metrics import *
from Unet import *
from image_processing import *
from image_augmentation import *
from F1_metrics import *
#from data_context import *
from data_extraction import *
from prediction import *
#from keras_pred import *
from unet_pred import *
from nyUnet import *

import code
import tensorflow.python.platform

import numpy as np

import tensorflow as tf
from scipy import misc, ndimage
import shutil

import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from pathlib import Path
from sklearn.utils import class_weight, shuffle


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
TESTING_SIZE = 50
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 10
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000
MAX_AUG = 1
NEW_DIM_TRAIN = 400

# The size of the patches each image is split into. Should be a multiple of 4, and the image
# size would be a multiple of this. For this assignment to get the delivery correct it has to be 16
IMG_PATCH_SIZE = 16


# Extract data into numpy arrays, divided into patches of 16x16
data_dir = '/content/drive/My Drive/data/'
train_data_filename = data_dir + 'training/images/'
train_labels_filename = data_dir + 'training/groundtruth/' 
test_data_filename = data_dir + 'test_set_images'

# Directive for storing the augmented training images
imgDir = 'training/augmented/images'
groundTruthDir = 'training/augmented/groundtruth'



#earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1)






Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
#x_train, y_train, x_test = load_data_img(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)
x_train, y_train, x_test = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)


print(y_train.shape)
print(x_train.shape)

Test data shape:  (50, 608, 608, 3)
Train data shape:  (100, 400, 400, 3)
Train labels shape:  (100, 400, 400, 2)
Number of samples in class 1 (background):  12395842
Number of samples in class 2 (road):  3604158 

(100, 400, 400, 2)
(100, 400, 400, 3)


In [5]:
# Class weigths
#classes = np.array([0,1])
#class_weights = class_weight.compute_class_weight('balanced',classes,y_train[:,1])
yweight = y_train[:,0,:,:]
yweight = yweight.flatten()
print(np.unique(yweight), sum(yweight))
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(yweight),
                                                 yweight)

print('Class weights: ',class_weights) 

[0. 1.] 40000.0
Class weights:  [1. 1.]


In [6]:
#model = ZF_UNET_224(class_weights,NEW_DIM_TRAIN)
inputs = Input((NEW_DIM_TRAIN, NEW_DIM_TRAIN,INPUT_CHANNELS))
model = create_model(inputs)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 400, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 400, 400, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 400, 400, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
# Checkpoint
filepath="/content/drive/My Drive/weightsUnet/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [9]:
# Train the model
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks_list,
          class_weight = class_weights
          )

X (100, 400, 400, 3) y (100, 400, 400, 2)
Train on 90 samples, validate on 10 samples
Epoch 1/10
90/90 [==============================] - 297s 3s/step - loss: 0.6046 - acc: 0.6892 - val_loss: 0.5306 - val_acc: 0.7641

Epoch 00001: val_acc improved from -inf to 0.76413, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 2/10
90/90 [==============================] - 285s 3s/step - loss: 0.5624 - acc: 0.7210 - val_loss: 0.5224 - val_acc: 0.7664

Epoch 00002: val_acc improved from 0.76413 to 0.76644, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 3/10
90/90 [==============================] - 285s 3s/step - loss: 0.5233 - acc: 0.7564 - val_loss: 0.6174 - val_acc: 0.6545

Epoch 00003: val_acc did not improve from 0.76644
Epoch 4/10
90/90 [==============================] - 285s 3s/step - loss: 0.4830 - acc: 0.7701 - val_loss: 0.5655 - val_acc: 0.7001

Epoch 00004: val_acc did not improve from 0.76644
Epoch 5/10
90/90 [=====================

In [10]:
#y_train_val = model.predict_classes(x_train)
#tp, tn, fp, fn = f1_values(y_train, y_train_val)
#f1 = f1_measure(tp, fp, fn)
#print("f1", f1)

prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
y_submit = np.zeros((((608//IMG_PATCH_SIZE)**2)*TESTING_SIZE,2))
for i in range(1,TESTING_SIZE+1):
  y_submit[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:], gtimg = get_pred_and_ysubmit_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, NEW_DIM_TRAIN,IMG_PATCH_SIZE,prediction_test_dir)


print('y_submit: ', y_submit.shape)
print('antall vei / antall bakgrunn: ', np.sum(y_submit[:,1]))

/content/drive/My Drive/data/test_set_images/test_1/test_1.png
/content/drive/My Drive/data/test_set_images/test_2/test_2.png
/content/drive/My Drive/data/test_set_images/test_3/test_3.png
/content/drive/My Drive/data/test_set_images/test_4/test_4.png
/content/drive/My Drive/data/test_set_images/test_5/test_5.png
/content/drive/My Drive/data/test_set_images/test_6/test_6.png
/content/drive/My Drive/data/test_set_images/test_7/test_7.png
/content/drive/My Drive/data/test_set_images/test_8/test_8.png
/content/drive/My Drive/data/test_set_images/test_9/test_9.png
/content/drive/My Drive/data/test_set_images/test_10/test_10.png
/content/drive/My Drive/data/test_set_images/test_11/test_11.png
/content/drive/My Drive/data/test_set_images/test_12/test_12.png
/content/drive/My Drive/data/test_set_images/test_13/test_13.png
/content/drive/My Drive/data/test_set_images/test_14/test_14.png
/content/drive/My Drive/data/test_set_images/test_15/test_15.png
/content/drive/My Drive/data/test_set_image

In [11]:
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE+1):
    oimg, imgpred = get_prediction_with_overlay_pixelwise(train_data_filename, i, 'train', model, PIXEL_DEPTH, NEW_DIM_TRAIN)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    imgpred.save(prediction_training_dir + "predictimg_" + str(i) + ".png")

(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 400, 400)
(1, 40

In [12]:

 

# Make submission file
prediction_to_submission2('submission_keras.csv', y_submit)

(72200, 2)
